In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [2]:
## data ingesation- from the website we need to scrape data

from langchain_community.document_loaders import WebBaseLoader

/Users/prakharchandrakar/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
loader = WebBaseLoader("https://openai.com/index/gpt-5-2-for-science-and-math/")
loader

In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://openai.com/index/gpt-5-2-for-science-and-math/', 'title': 'Advancing science and math with GPT-5.2 | OpenAI', 'description': 'GPT-5.2 is our strongest model yet for math and science work.', 'language': 'en-US'}, page_content='Advancing science and math with GPT-5.2 | OpenAISkip to main contentLog inSwitch toChatGPT(opens in a new window)Sora(opens in a new window)API Platform(opens in a new window)ResearchSafetyFor BusinessFor DevelopersChatGPTSoraStoriesCompanyNewsResearchBack to main menuResearch IndexResearch OverviewResearch ResidencyOpenAI for ScienceLatest AdvancementsGPT-5.2GPT-5.1Sora 2GPT-5OpenAI o3 and o4-miniGPT-4.5SafetyBack to main menuSafety ApproachSecurity & PrivacyFor BusinessBack to main menuBusiness OverviewSolutionsLearnStartupsChatGPT PricingAPI PricingContact SalesFor DevelopersBack to main menuAPI PlatformAPI PricingAgentsCodexOpen ModelsCommunity(opens in a new window)ChatGPTBack to main menuExplore ChatGPTBusinessEnterpris

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000 ,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [6]:
from langchain_openai import OpenAIEmbeddings
embadding = OpenAIEmbeddings()

In [7]:
from langchain_community.vectorstores import FAISS
vectorstoredb= FAISS.from_documents(documents,embadding)

In [8]:
vectorstoredb

In [9]:
### quary from vector store db
query = "Stronger performance where precision matters"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'UsOur CharterFoundationCareersBrand GuidelinesNewsLog inOpenAITable of contentsStronger performance where precision mattersLooking aheadDecember 11, 2025PublicationProductAdvancing science and math with GPT-5.2GPT‑5.2 is our strongest model yet for math and science work.Read the paper(opens in a new window)Loading…ShareOne of our hopes for strong AI is that it will accelerate scientific research for the benefit of everyone, helping researchers explore more ideas, test them faster, and turn discoveries into impact.\xa0Over the past year, we’ve been working closely with scientists across math, physics, biology, and computer science to understand where AI can help—and where it still falls short. Last month, we published a paper\u2060 that compiles early case studies across math, physics, biology, computer science, astronomy, and materials science in which GPT‑5 helped researchers showing how GPT‑5 has already begun contributing to real scientific work. With GPT‑5.2, we’re starting to see

In [10]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:

<context>   
{context}
</context>

Question: {question}
"""
)

document_chain = prompt | llm | StrOutputParser()
document_chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n\n<context>   \n{context}\n</context>\n\nQuestion: {question}\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x32880eec0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0

In [12]:
from langchain_core.documents import Document

document_chain.invoke({
    "question": "Stronger performance where precision matters",
    "context": [
        Document(page_content="Stronger performance where precision matters")
    ]
})




"The document's content suggests that there is a focus on enhancing or improving performance in areas or situations where precision is a critical factor. This likely means that the document discusses improvements or strategies for achieving better results in tasks or fields where accuracy and careful attention to detail are essential."

In [16]:
from langchain_core.output_parsers import StrOutputParser

# document chain
document_chain = prompt | llm | StrOutputParser()

# retriever
retriever = vectorstoredb.as_retriever()

# FIXED retrieval chain
retrieval_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],
    }
    | document_chain
)

response = retrieval_chain.invoke({
    "question": "Stronger performance where precision matters"
})

print(response)


GPT-5.2 demonstrates stronger performance where precision is critical, particularly in math and science work. It is essential for models to have strong mathematical reasoning to ensure reliability in scientific and technical work. This allows the model to follow multi-step logic, maintain consistency in quantities, and avoid subtle errors that can occur in real analyses, such as simulations, statistics, forecasting, and modeling. Improvements observed in benchmarks like FrontierMath indicate not just a specific skill but stronger general reasoning and abstraction, which are crucial for scientific workflows, including coding, data analysis, and experimental design. These characteristics contribute to progress toward general intelligence by providing broad, transferable reasoning skills necessary across various domains in science, engineering, and real-world decision-making.


In [19]:
## Get the response form the LLM
response=retrieval_chain.invoke({"question":"Stronger performance where precision matters"})
print(response)

The context outlines that GPT-5.2, particularly through its Pro and Thinking variants, excels in situations where precision is crucial. This enhanced precision is supported by strong mathematical reasoning, enabling the models to maintain consistency, accurately follow multi-step logic, and avoid errors in analyses like simulations, statistics, forecasting, and modeling. Furthermore, improvements in benchmarks such as FrontierMath indicate that these capabilities reflect stronger general reasoning and abstraction skills, which are directly applicable to scientific workflows like coding, data analysis, and experimental design. These qualities also contribute to progress toward general intelligence, as the models exhibit broad, transferable reasoning skills essential for scientific, engineering, and real-world decision-making.
